# <font style="color:blue">Spojíme je všechno dohromady: Pipeline LeNet5 s Trainerem</font>

Podívejme se na to, jak můžeme sestavit trénovací pipeline pomocí pomocné třídy Trainer a dalších pomocných tříd, o kterých jsme v tomto notebooku hovořili dříve.
Importujte všechny potřebné třídy a funkce:

In [1]:
!wget "https://raw.githubusercontent.com/RadimKozl/OpenCV_academy_my_work/refs/heads/main/work_module/trainer.zip" -O ./trainer.zip

!ls /kaggle/working/

!unzip /kaggle/working/trainer.zip

!rm /kaggle/working/trainer.zip

--2024-10-02 21:49:01--  https://raw.githubusercontent.com/RadimKozl/OpenCV_academy_my_work/refs/heads/main/work_module/trainer.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8348 (8.2K) [application/zip]
Saving to: './trainer.zip'

./trainer.zip       100%[===================>]   8.15K  --.-KB/s    in 0s      

2024-10-02 21:49:02 (50.6 MB/s) - './trainer.zip' saved [8348/8348]

trainer.zip
Archive:  /kaggle/working/trainer.zip
  inflating: trainer/__init__.py     
  inflating: trainer/base_metric.py  
  inflating: trainer/configuration.py  
  inflating: trainer/hooks.py        
  inflating: trainer/metrics.py      
  inflating: trainer/tensorboard_visualizer.py  
  inflating: trainer/trainer.py      
  inflating: trainer/utils.py        
  inflating: 

In [2]:
ls /kaggle/working/

trainer/


In [3]:
!unzip /kaggle/working/trainer.zip

unzip:  cannot find or open /kaggle/working/trainer.zip, /kaggle/working/trainer.zip.zip or /kaggle/working/trainer.zip.ZIP.


In [4]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

from operator import itemgetter

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms
from torch.optim.lr_scheduler import MultiStepLR

torch.multiprocessing.set_start_method('spawn', force=True)

from trainer import Trainer, hooks, configuration
from trainer.utils import setup_system, patch_configs
from trainer.metrics import AccuracyEstimator
from trainer.tensorboard_visualizer import TensorBoardVisualizer

## <font style="color:Green">1. Získejme nahrávání dat pro Trénovací a Validační data</font>

Definujeme wrapper dat a transformace (stejnou cestou jako předtím):

In [5]:
def get_data(batch_size, data_root='data', num_workers=1):

    train_test_transforms = transforms.Compose([
        # Resize to 32X32
        transforms.Resize((32, 32)),
        # this re-scales image tensor values between 0-1. image_tensor /= 255
        transforms.ToTensor(),
        # subtract mean (0.1307) and divide by variance (0.3081).
        # This mean and variance is calculated on training data (verify yourself)
        transforms.Normalize((0.1307, ), (0.3081, ))
    ])

    # train dataloader
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST(root=data_root, train=True, download=True, transform=train_test_transforms),
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers
    )

    # test dataloader
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(root=data_root, train=False, download=True, transform=train_test_transforms),
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers
    )
    return train_loader, test_loader

## <font style="color:Green">2. Definujeme model</font>

Definujte model (stejným způsobem jako dříve):

In [6]:
class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()

        self._body = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )
        self._head = nn.Sequential(
            nn.Linear(in_features=16 * 5 * 5, out_features=120), nn.ReLU(inplace=True),
            nn.Linear(in_features=120, out_features=84), nn.ReLU(inplace=True),
            nn.Linear(in_features=84, out_features=10)
        )

    def forward(self, x):
        x = self._body(x)
        x = x.view(x.size()[0], -1)
        x = self._head(x)
        return x

## <font style="color:Green">3. Spusťte Experiment / Trénink</font>

Definujte Experiment s daným modelem a danými daty. Je to opět stejná myšlenka: ponecháme věci, u kterých je méně pravděpodobné, že se změní, uvnitř objektu a nakonfigurujeme jej s věcmi, u kterých je pravděpodobnější, že se změní.

Možná se divíte, proč vkládáme konkrétní metriku a optimalizátor do kódu experimentu a neurčujeme je jako parametry. Ale třída experiment je jen praktický způsob, jak uložit všechny části experimentu na jednom místě. Pokud změníte ztrátovou funkci, optimalizátor nebo model - vypadá to jako další experiment, že? Zaslouží si tedy být samostatnou třídou.

Vnitřní struktura třídy Trainer je o něco komplikovanější ve srovnání s tím, o čem jsme hovořili výše – jde pouze o to, abychom se dokázali vyrovnat s různými druhy úkolů, o kterých budeme v tomto kurzu diskutovat. V následujících přednáškách se o vnitřní struktuře Traineru dozvíme trochu více a nyní se podíváme na to, jak kompaktní a sebepopisný kód je:

In [7]:
class Experiment:
    def __init__(
        self,
        system_config: configuration.SystemConfig = configuration.SystemConfig(),
        dataset_config: configuration.DatasetConfig = configuration.DatasetConfig(),
        dataloader_config: configuration.DataloaderConfig = configuration.DataloaderConfig(),
        optimizer_config: configuration.OptimizerConfig = configuration.OptimizerConfig()
    ):
        self.loader_train, self.loader_test = get_data(
            batch_size=dataloader_config.batch_size,
            num_workers=dataloader_config.num_workers,
            data_root=dataset_config.root_dir
        )
        
        setup_system(system_config)

        self.model = LeNet5()
        self.loss_fn = nn.CrossEntropyLoss()
        self.metric_fn = AccuracyEstimator(topk=(1, ))
        self.optimizer = optim.SGD(
            self.model.parameters(),
            lr=optimizer_config.learning_rate,
            weight_decay=optimizer_config.weight_decay,
            momentum=optimizer_config.momentum
        )
        self.lr_scheduler = MultiStepLR(
            self.optimizer, milestones=optimizer_config.lr_step_milestones, gamma=optimizer_config.lr_gamma
        )
        self.visualizer = TensorBoardVisualizer()

    def run(self, trainer_config: configuration.TrainerConfig) -> dict:

        device = torch.device(trainer_config.device)
        self.model = self.model.to(device)
        self.loss_fn = self.loss_fn.to(device)

        model_trainer = Trainer(
            model=self.model,
            loader_train=self.loader_train,
            loader_test=self.loader_test,
            loss_fn=self.loss_fn,
            metric_fn=self.metric_fn,
            optimizer=self.optimizer,
            lr_scheduler=self.lr_scheduler,
            device=device,
            data_getter=itemgetter(0),
            target_getter=itemgetter(1),
            stage_progress=trainer_config.progress_bar,
            get_key_metric=itemgetter("top1"),
            visualizer=self.visualizer,
            model_saving_frequency=trainer_config.model_saving_frequency,
            save_dir=trainer_config.model_dir
        )

        model_trainer.register_hook("end_epoch", hooks.end_epoch_hook_classification)
        self.metrics = model_trainer.fit(trainer_config.epoch_num)
        return self.metrics

In [8]:
def main():
    '''Run the experiment
    '''
    # patch configs depending on cuda availability
    dataloader_config, trainer_config = patch_configs(epoch_num_to_set=15)
    dataset_config = configuration.DatasetConfig(root_dir="data")
    experiment = Experiment(dataset_config=dataset_config, dataloader_config=dataloader_config)
    results = experiment.run(trainer_config)

    return results

In [9]:
if __name__ == '__main__':
    main()

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 34739673.11it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1009607.14it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



...

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/240 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/240 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

Takže v několika řádcích kódu jsme získali robustnější systém, než jsme měli předtím – máme bohatší vizualizace, lépe konfigurovatelný tréninkový proces a oddělili jsme pipeline pro trénink od modelu – takže se můžeme soustředit na věci, na kterých záleží nejvíc.

# <font style="color:blue">Reference</font>

Možná se ptáte, zda je to běžný způsob hlubokého učení, nebo to tady přeháníme. Můžeme vás ujistit, že toto je běžný způsob, jak se provádí výzkum hlubokého učení v průmyslu – většina společností a výzkumných skupin investuje do budování těchto rámců školení DL pro své projekty a některé z nich jsou dokonce publikovány jako open-source. Abych jmenoval pár z nich:

- https://github.com/NVlabs/SPADE
- https://github.com/pytorch/ignite
- https://github.com/PyTorchLightning/pytorch-lightning
- https://github.com/catalyst-team/catalyst
- https://github.com/open-mmlab/mmdetection
- https://github.com/fastai/fastai